# Statista data
Statista Consumer Insights: https://www.statista.com/global-consumer-survey 

In [9]:
import pandas as pd
import plotly.express as px
import glob
import numpy as np
from io import StringIO
from typing import List

from plotly.subplots import make_subplots
from plotly import graph_objects as go

In [10]:
data_root = '../data/statista/'

file_lst = glob.glob(data_root + '*.csv')


In [11]:
def parse_statista_csv(filepath: str) -> pd.DataFrame:
    """Reads a csv file downloaded from statista.com and returns a pandas dataframe

    Args:
        filepath (str): Filepath of the csv file

    Returns:
        pd.DataFrame: Pandas dataframe with the data
    """
    with open(filepath, 'r') as f:
        lines = f.readlines()
    # print question:
    print(f'Question: {lines[4]}')
    update_indices = [idx for idx, line in enumerate(lines) if ' - Update' in line]
    # create pairs from the above list
    pairs = list(zip(update_indices, update_indices[1:]))
    # Filter out short pairse because they do not contain any data
    pairs = [pair for pair in pairs if abs(pair[1] - pair[0]) > 4]
    df_lst = []
    for update_start, update_end in pairs:
        year, update = lines[update_start].strip().replace('Update ', '').replace('"', '').split('-')
        # print(f'Processing {year} {update}')
        year = int(year)
        update = int(update)
        start_idx = update_start + 2
        end_idx = update_end - 2
        csv_str = ''.join(lines[start_idx:end_idx])
        # print(csv_str)
        
        df = pd.read_csv(StringIO(csv_str), sep=';', header=None).T

        # Filter rows based on column: 1
        df = df[(df[1] == "absolute") | (df[1].isna())]

        # Drop column: '1'
        df = df.drop(columns=[1])
        df.loc[0, 0] = 'Country'
        df.columns = df.iloc[0]
        df = df[1:]
        df['Year'] = year
        df['Update'] = update
        df.set_index(['Year', 'Update', 'Country'], inplace=True)

        for col in df.columns:
            # float to be able to assign null
            df[col] = df[col].str.split(' / ').str[0].str.replace(',', '').astype(float)
        df_lst.append(df)
        for col in df.columns:
            if col != 'Base':
                df[col] = df[col] / df['Base'] * 100
                
    df = pd.concat(df_lst)
    country_lst = ['Finland', 'France', 'Italy', 'Spain', 'Germany', 'Switzerland', 'Sweden', 'Austria', 'Poland']
    df = df.query('Country in @country_lst')

    return df

In [12]:
def create_bar_plot(df: pd.DataFrame, title: str, cols_to_plot: List[str] = None, barmode: str = 'group') -> go.Figure:
    plot_df = df.copy()
    plot_df.reset_index(inplace=True)
    if cols_to_plot is None:
        cols_to_plot = plot_df.columns.tolist()[4:]
    max_cols = 4
    n_rows = int(np.ceil(len(plot_df['Country'].unique())/max_cols))
    plot_titles = plot_df['Country'].unique()
    fig = make_subplots(rows=n_rows, cols=max_cols, subplot_titles=plot_titles, shared_yaxes=True, shared_xaxes=True)
    fig.update_layout(title=title, legend_title='Answer')
    cur_row = 1
    cur_col = 1
    first = True
    color_palette = px.colors.qualitative.Dark24
    if cols_to_plot is None:
        cols_to_plot = plot_df.columns.tolist()[4:]
    for country in plot_df['Country'].unique():
        df = plot_df.query('Country == @country')
        for i, col in enumerate(cols_to_plot):
            fig.add_trace(go.Bar(x=df['Year'], y=df[col], name=col, marker=dict(color=color_palette[i]), showlegend=first, legendgroup=col), row=cur_row, col=cur_col)
            fig.update_xaxes(title_text="Year", row=cur_row, col=cur_col)
            fig.update_yaxes(title_text="Percentage", row=cur_row, col=cur_col)
        first = False
        cur_col += 1
        if cur_col > max_cols:
            cur_row += 1
            cur_col = 1
    layout_dict = {}
    for i in range(len(plot_df['Country'].unique())):
        if i == 0:
            layout_dict['xaxis_showticklabels'] = True
            layout_dict['yaxis_showticklabels'] = True
        else:
            layout_dict[f'xaxis{i}_showticklabels'] = True
            layout_dict[f'yaxis{i}_showticklabels'] = True
    fig.update_layout(legend_title='Answer', barmode=barmode, margin=dict(l=20, r=20, t=100, b=20), height=600)
    fig.update_layout(layout_dict)

    return fig

In [13]:
def create_scatter_plot(df: pd.DataFrame, title: str, cols_to_plot: List[str] = None) -> go.Figure:
    plot_df = df.copy()
    plot_df.reset_index(inplace=True)
    # make multiple line subplots for each country
    max_cols = 4
    n_rows = int(np.ceil(len(plot_df['Country'].unique())/max_cols))
    fig = make_subplots(rows=n_rows, cols=max_cols, subplot_titles=plot_df['Country'].unique(), shared_yaxes=True, shared_xaxes=True)
    fig.update_layout(title=title, legend_title='Answer')
    cur_row = 1
    cur_col = 1
    first = True
    new_row = True
    color_palette = px.colors.qualitative.Dark24
    if cols_to_plot is None:
        cols_to_plot = plot_df.columns.tolist()[4:]
    for country in plot_df['Country'].unique():
        df = plot_df.query('Country == @country')
        for i, col in enumerate(cols_to_plot):
            fig.add_trace(
                go.Scatter(x=df['Year'], y=df[col], name=col, mode='markers+lines', marker=dict(color=color_palette[i]), showlegend=first, legendgroup=col),
                row=cur_row, col=cur_col)
            fig.update_xaxes(title_text="Year", row=cur_row, col=cur_col)
            if new_row:
                fig.update_yaxes(title_text="Percentage", row=cur_row, col=cur_col)

        first = False
        new_row = False
        cur_col += 1
        if cur_col > max_cols:
            new_row = True
            cur_row += 1
            cur_col = 1

    layout_dict = {}
    for i in range(len(plot_df['Country'].unique())):
        if i == 0:
            layout_dict['xaxis_showticklabels'] = True
            layout_dict['yaxis_showticklabels'] = True
        else:
            layout_dict[f'xaxis{i}_showticklabels'] = True
            layout_dict[f'yaxis{i}_showticklabels'] = True
    fig.update_layout(layout_dict)

    fig.update_layout(margin=dict(l=20, r=20, t=100, b=20), height=600)
    return fig

## Attitude towards mobility

In [14]:
mobility_attitude_df = parse_statista_csv('../data/statista/Attitudes towards mobility.csv')
# filter for update 1
mobility_attitude_df = mobility_attitude_df.query('Update == 1')
mobility_attitude_df.info()

Question: Which of these statements about cars and mobility do you agree with? (multi-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 54 entries, (2018, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 17 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   Base                                                                             54 non-null     float64
 1   A car is just a means of transportation                                          9 non-null      float64
 2   A car reflects its owner's personality                                           18 non-null     float64
 3   Cars represent independence and freedom                                          9 non-null      float64
 4   Driving cars is bad for the environment                  

In [15]:
mobility_attitude_df

0                          Base  A car is just a means of transportation  \
Year Update Country                                                        
2018 1      Austria      2044.0                                53.816047   
            Finland      1044.0                                46.551724   
            France       2041.0                                42.381186   
            Germany      2080.0                                43.317308   
            Italy        2074.0                                37.560270   
            Poland       1041.0                                43.419789   
            Spain        2037.0                                32.106038   
            Sweden       1039.0                                39.172281   
            Switzerland  1727.0                                47.944412   
2019 1      Austria      2072.0                                      NaN   
            Finland      1043.0                                      NaN   
            France       2067.0                                      NaN   
            Germany      2087.0                                      NaN   
            Italy        2088.0                                      NaN   
            Poland       1048.0                                      NaN   
            Spain        2058.0                                      NaN   
            Sweden       1052.0                                      NaN   
            Switzerland  1057.0                                      NaN   
2020 1      Austria      2090.0                                      NaN   
            Finland      1042.0                                      NaN   
            France       2095.0                                      NaN   
            Germany      2097.0                                      NaN   
            Italy        2103.0                                      NaN   
            Poland       2091.0                                      NaN   
            Spain        2094.0                                      NaN   
            Sweden       2100.0                                      NaN   
            Switzerland  1062.0                                      NaN   
2021 1      Austria      2086.0                                      NaN   
            Finland      2088.0                                      NaN   
            France       2096.0                                      NaN   
            Germany      2613.0                                      NaN   
            Italy        2097.0                                      NaN   
            Poland       2096.0                                      NaN   
            Spain        2093.0                                      NaN   
            Sweden       2087.0                                      NaN   
            Switzerland  2074.0                                      NaN   
2022 1      Austria      2035.0                                      NaN   
            Finland      2032.0                                      NaN   
            France       2033.0                                      NaN   
            Germany      4525.0                                      NaN   
            Italy        2032.0                                      NaN   
            Poland       2034.0                                      NaN   
            Spain        2029.0                                      NaN   
            Sweden       2035.0                                      NaN   
            Switzerland  2012.0                                      NaN   
2023 1      Austria      2035.0                                      NaN   
            Finland      2034.0                                      NaN   
            France       2035.0                                      NaN   
            Germany      6026.0                                      NaN   
            Italy        2031.0                                      NaN   
            Poland       2033.0                 

In [16]:
fig = create_bar_plot(mobility_attitude_df, 'Attitudes towards mobility')
fig.update_layout(height=900)
fig.show()

In [17]:
fig = create_scatter_plot(mobility_attitude_df, 'Attitudes towards mobility')
fig.update_layout(height=900)
fig.show()

In [18]:
# commute_countries = commute_duration_df.reset_index()['Country'].unique().tolist()
# frequent_modes_of_transport_countries = frequent_modes_of_transport_df.reset_index()['Country'].unique().tolist()
# commute_mode_countries = commute_modes_of_transport_df.reset_index()['Country'].unique().tolist()
# transportation_usage_public_countries = frequency_public_transport_df.reset_index()['Country'].unique().tolist()
# transportation_usage_cars_countries = frequency_car_df.reset_index()['Country'].unique().tolist()
# transportation_usage_combined_countries = frequency_combined_df.reset_index()['Country'].unique().tolist()
# # find common elements
# common_countries = list(set(commute_countries) & set(frequent_modes_of_transport_countries) & set(commute_mode_countries) & set(transportation_usage_public_countries) & set(transportation_usage_cars_countries) & set(transportation_usage_combined_countries))
# common_countries

## Duration of daily commute

In [19]:
commute_duration_df = parse_statista_csv('../data/statista/Duration of daily commute.csv')
# filter for update 1
commute_duration_df = commute_duration_df.query('Update == 1')
commute_duration_df.info()

Question: On average, how long is your daily commute to work/school/university (one way)? (single-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 45 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Base                  45 non-null     float64
 1   Less than 15 minutes  45 non-null     float64
 2   15 to 29 minutes      45 non-null     float64
 3   30 to 59 minutes      45 non-null     float64
 4   60 to 119 minutes     45 non-null     float64
 5   120 minutes and more  45 non-null     float64
 6   I do not commute      45 non-null     float64
dtypes: float64(7)
memory usage: 3.9+ KB


In [20]:
commute_duration_df

0                          Base  Less than 15 minutes  15 to 29 minutes  \
Year Update Country                                                       
2019 1      Austria      1050.0             20.571429         27.904762   
            Finland      1043.0             24.065197         30.393097   
            France       1047.0             26.647564         32.760267   
            Germany      1047.0             17.956065         32.378223   
            Italy        1051.0             29.019981         35.394862   
            Poland       1048.0             23.950382         35.209924   
            Spain        1040.0             27.692308         39.807692   
            Sweden       1052.0             24.239544         30.323194   
            Switzerland  1057.0             21.570482         30.274361   
2020 1      Austria      2090.0             21.818182         29.138756   
            Finland      1042.0             26.391555         32.821497   
            France       2095.0             27.541766         33.031026   
            Germany      2097.0             19.885551         31.139723   
            Italy        2103.0             29.671897         36.329054   
            Poland       2091.0             20.994739         37.254902   
            Spain        2094.0             28.414518         35.912130   
            Sweden       2100.0             24.761905         30.380952   
            Switzerland  1062.0             21.751412         33.239171   
2021 1      Austria      2086.0             20.421860         28.571429   
            Finland      2088.0             27.634100         33.141762   
            France       2096.0             25.333969         34.494275   
            Germany      2613.0             19.556066         30.884041   
            Italy        2097.0             28.946113         34.191702   
            Poland       2096.0             21.230916         38.406489   
            Spain        2093.0             29.240325         37.028189   
            Sweden       2087.0             22.568280         31.288931   
            Switzerland  2074.0             21.166827         31.774349   
2022 1      Austria      2035.0             19.852580         28.943489   
            Finland      2032.0             26.476378         34.448819   
            France       2033.0             25.774717         33.251353   
            Germany      4525.0             18.718232         30.872928   
            Italy        2032.0             25.147638         36.269685   
            Poland       2034.0             20.304818         34.464110   
            Spain        2029.0             26.269098         39.773287   
            Sweden       2035.0             21.719902         31.154791   
            Switzerland  2012.0             22.514911         31.013917   
2023 1      Austria      2035.0             22.260442         29.041769   
            Finland      2034.0             26.941986         33.874140   
            France       2035.0             23.439803         34.643735   
            Germany      6026.0             18.187853         32.791238   
            Italy        2031.0             26.686361         38.109306   
            Poland       2033.0             17.117560         36.104279   
            Spain        2029.0             27.501232         38.590439   
            Sweden       2030.0             23.152709         30.492611   
            Switzerland  1021.0             17.923604         32.713026   

0                        30 to 59 minutes  60 to 119 minutes  \
Year Update Country                                            
2019 1      Austria             22.571429           7.047619   
            Finland             20.134228           4.506232   
            France              19.006686           4.202483   
            Germany             20.821394           4.680038   
            Italy               18.363463           5.233111   
            Poland        

In [21]:
fig = create_bar_plot(commute_duration_df, 'Duration of daily commute', barmode='stack')
fig.update_layout(height=1000)
fig.show()

In [22]:
fig = create_bar_plot(commute_duration_df, 'Duration of daily commute', barmode='group')
fig.update_layout(height=1000)
fig.show()

In [23]:
fig = create_scatter_plot(commute_duration_df, 'Duration of daily commute')
fig.update_layout(height=1000)
fig.show()


## Frequent users of modes of transportation

In [24]:
frequent_modes_of_transport_df = parse_statista_csv('../data/statista/Frequent users of modes of transportation.csv')
frequent_modes_of_transport_df = frequent_modes_of_transport_df.query('Update == 1')
frequent_modes_of_transport_df.info()

Question: Recode based on Transportation usage frequency (multi-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 15 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   Base                                                                           43 non-null     float64
 1   Bike rentals / bike sharing                                                    43 non-null     float64
 2   Car sharing (short periods, e.g., by the minute, not traditional car rental)   43 non-null     float64
 3   Local public transportation                                                    43 non-null     float64
 4   Own bicycle                                                                    43 non-null 

In [25]:
create_bar_plot(frequent_modes_of_transport_df, 'Frequent users of modes of transportation', barmode='group')

In [26]:
fig = create_bar_plot(frequent_modes_of_transport_df, 'Frequent users of modes of transportation', barmode='stack')
fig.update_layout(height=1000)
fig.show()

In [27]:
fig = create_scatter_plot(frequent_modes_of_transport_df, 'Frequent users of modes of transportation')
# fig.update_layout(height=700)
fig.show()

## Modes of transportation for commuting

In [28]:
commute_modes_of_transport_df = parse_statista_csv('../data/statista/Modes of transportation for commuting.csv')
commute_modes_of_transport_df = commute_modes_of_transport_df.query('Update == 1')
commute_modes_of_transport_df.info()

Question: Which modes of transportation do you use for your daily commute to work/school/university? (multi-pick) Base: respondents who commute (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 15 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   Base                                                                           43 non-null     float64
 1   Bike rentals / bike sharing                                                    43 non-null     float64
 2   Car sharing (short periods, e.g., by the minute, not traditional car rental)   43 non-null     float64
 3   Own / household car                                                            43 non-null     float64
 4   Own bicycle                            

In [29]:
create_bar_plot(commute_modes_of_transport_df, 'Modes of transportation for commuting', barmode='group')

In [30]:
fig = create_bar_plot(commute_modes_of_transport_df, 'Modes of transportation for commuting', barmode='stack')
fig.update_layout(height=1000)

fig.show()

In [31]:
create_scatter_plot(commute_modes_of_transport_df, 'Modes of transportation for commuting')

## Bus user (local)

In [32]:
bus_user_df = parse_statista_csv('../data/statista/Bus user (local).csv')
bus_user_df = bus_user_df.query('Update == 1')
bus_user_df.info()

Question: Recode based on Mobility service usage (detailed) (single-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 45 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Base              45 non-null     float64
 1   Bus user (local)  45 non-null     float64
 2   Non-user          45 non-null     float64
dtypes: float64(3)
memory usage: 2.4+ KB


In [33]:
# fig = create_bar_plot(bus_user_df, 'Bus user (local)', barmode='group')
# fig.update_layout(height=900)
# fig.show()

In [34]:
# fig = create_bar_plot(bus_user_df, 'Bus user (local)', barmode='stack')
# fig.update_layout(height=900)
# fig.show()

In [35]:
fig = create_scatter_plot(bus_user_df, 'Bus user (local)')
fig.update_layout(height=900)
fig.show()

## Train user (local)

In [36]:
train_user_df = parse_statista_csv('../data/statista/Train user (local).csv')
train_user_df = train_user_df.query('Update == 1')
train_user_df.info()

Question: Recode based on Mobility service usage (detailed) (single-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 45 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Base                45 non-null     float64
 1   Train user (local)  45 non-null     float64
 2   Non-user            45 non-null     float64
dtypes: float64(3)
memory usage: 2.4+ KB


In [37]:
# fig = create_bar_plot(train_user_df, 'Train user (local)', barmode='group')
# fig.update_layout(height=900)
# fig.show()

In [38]:
# fig = create_bar_plot(train_user_df, 'Train user (local)', barmode='stack')
# fig.update_layout(height=900)
# fig.show()

In [39]:
fig = create_scatter_plot(train_user_df, 'Train user (local)')
fig.update_layout(height=900)
fig.show()

## Transportation usage frequency Local public transportation

In [56]:
frequency_public_transport_df = parse_statista_csv('../data/statista/Transportation usage frequency Local public transportation.csv')
frequency_public_transport_df = frequency_public_transport_df.query('Update == 1')
frequency_public_transport_df.info()

Question: How often do you use the following modes of transportation? (single-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Base                 43 non-null     float64
 1   (Almost) daily       43 non-null     float64
 2   2-5 times per week   43 non-null     float64
 3   1-4 times per month  43 non-null     float64
 4   Every 2-3 months     43 non-null     float64
 5   Less often           43 non-null     float64
 6   Not at all           43 non-null     float64
dtypes: float64(7)
memory usage: 3.4+ KB


In [57]:
create_bar_plot(frequency_public_transport_df, 'Transportation usage frequency Local public transportation', barmode='stack')

In [42]:
create_bar_plot(frequency_public_transport_df, 'Transportation usage frequency: Local public transportation', barmode='group')

In [43]:
fig = create_scatter_plot(frequency_public_transport_df, 'Transportation usage frequency: Local public transportation')
fig.update_layout(height=1000)
fig.show()

## Transportation usage frequency Own bicycle

In [44]:
frequency_bicycle_df = parse_statista_csv('../data/statista/Transportation usage frequency Own bicycle.csv')
frequency_bicycle_df = frequency_bicycle_df.query('Update == 1')
frequency_bicycle_df.info()

Question: How often do you use the following modes of transportation? (single-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Base                 43 non-null     float64
 1   (Almost) daily       43 non-null     float64
 2   2-5 times per week   43 non-null     float64
 3   1-4 times per month  43 non-null     float64
 4   Every 2-3 months     43 non-null     float64
 5   Less often           43 non-null     float64
 6   Not at all           43 non-null     float64
dtypes: float64(7)
memory usage: 3.4+ KB


In [45]:
create_bar_plot(frequency_bicycle_df, 'Transportation usage frequency: Own bicycle', barmode='stack')

In [46]:
create_bar_plot(frequency_bicycle_df, 'Transportation usage frequency: Own bicycle', barmode='group')

In [47]:
fig = create_scatter_plot(frequency_bicycle_df, 'Transportation usage frequency: Own bicycle')
fig.update_layout(height=700)
fig.show()

## Transportation usage frequency Own or household car

In [48]:
frequency_car_df = parse_statista_csv('../data/statista/Transportation usage frequency Own or household car.csv')
frequency_car_df = frequency_car_df.query('Update == 1')
frequency_car_df.info()


Question: How often do you use the following modes of transportation? (single-pick) Base: all respondents (Mobility, Travel)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43 entries, (2019, 1, 'Austria') to (2023, 1, 'Switzerland')
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Base                 43 non-null     float64
 1   (Almost) daily       43 non-null     float64
 2   2-5 times per week   43 non-null     float64
 3   1-4 times per month  43 non-null     float64
 4   Every 2-3 months     43 non-null     float64
 5   Less often           43 non-null     float64
 6   Not at all           43 non-null     float64
dtypes: float64(7)
memory usage: 3.4+ KB


In [49]:
create_bar_plot(frequency_car_df, 'Transportation usage frequency: Own or household car', barmode='stack')

In [50]:
create_bar_plot(frequency_car_df, 'Transportation usage frequency: Own or household car', barmode='group')

In [51]:
fig = create_scatter_plot(frequency_car_df, 'Transportation usage frequency: Own or household car')
fig.update_layout(height=1000)
fig.show()

## Compare frequency - public transport, bicycle and car

this is done by joining the required dataframes and plotting them. The columns to be plotted were limited to a subset to make the plots less cluttered.

In [52]:
frequency_combined_df = (
    frequency_public_transport_df.drop(columns="Base")
    .add_prefix("Public Transport ")
    .join(frequency_bicycle_df.drop(columns="Base").add_prefix("Bicycle "), how="outer")
    .join(frequency_car_df.drop(columns="Base").add_prefix("Car "), how="outer")
)
frequency_combined_df

0                        Public Transport (Almost) daily  \
Year Update Country                                        
2019 1      Austria                            21.238095   
            France                             10.601719   
            Germany                            13.849093   
            Italy                              11.512845   
            Poland                             14.408397   
            Spain                              16.250000   
            Sweden                             18.916350   
2020 1      Austria                            20.861244   
            Finland                            15.642994   
            France                              9.212411   
            Germany                            14.115403   
            Italy                              10.271041   
            Poland                             13.964610   
            Spain                              17.096466   
            Sweden                             15.714286   
            Switzerland                        21.374765   
2021 1      Austria                            18.216683   
            Finland                            10.057471   
            France                              8.062977   
            Germany                            10.486031   
            Italy                               7.916071   
            Poland                             10.400763   
            Spain                              12.326804   
            Sweden                             11.068519   
            Switzerland                        17.984571   
2022 1      Austria                            14.643735   
            Finland                             8.021654   
            France                              9.394983   
            Germany                             9.900552   
            Italy                               6.151575   
            Poland                             12.045231   
            Spain                              10.990636   
            Sweden                             10.565111   
            Switzerland                        19.085487   
2023 1      Austria                            16.167076   
            Finland                             8.062930   
            France                              8.697789   
            Germany                            11.782277   
            Italy                               8.468735   
            Poland                             12.395475   
            Spain                              13.208477   
            Sweden                             12.167488   
            Switzerland                        15.475024   

0                        Public Transport 2-5 times per week  \
Year Update Country                                            
2019 1      Austria                                11.238095   
            France                                  8.978032   
            Germany                                12.511939   
            Italy                                   8.372978   
            Poland                                 13.740458   
            Spain                                  14.711538   
            Sweden                                 12.357414   
2020 1      Austria                                11.770335   
            Finland                                11.708253   
            France                                  8.257757   
            Germany                                14.401526   
            Italy                                  12.410842   
            Poland                                 13.821138   
            Spain                                  16.953200   
            Sweden                                 14.142857   
            Switzerland                            17.984934   
2021 1      Austria                                12.655801   
            Finland                                12.787356   
            France 

In [53]:

# fig = create_scatter_plot(frequency_combined_df, 'Transportation usage frequency', cols_to_plot=[col for col in frequency_combined_df.columns.to_list() if 'daily' in col or 'Not at all' in col or '2-5 times per week' in col])
fig = create_scatter_plot(frequency_combined_df, 'Transportation usage frequency', cols_to_plot=[col for col in frequency_combined_df.columns.to_list() if 'daily' in col])
fig.update_layout(height=1000)
# fig.show(renderer='browser')

In [54]:
create_bar_plot(frequency_combined_df, 'Transportation usage frequency', cols_to_plot=[col for col in frequency_combined_df.columns.to_list() if 'daily' in col or 'Not at all' in col or '2-5 times per week' in col], barmode='stack')


In [55]:
fig = create_bar_plot(frequency_combined_df, 'Transportation usage frequency', cols_to_plot=[col for col in frequency_combined_df.columns.to_list() if 'daily' in col or 'Not at all' in col or '2-5 times per week' in col], barmode='group')
fig.show()